In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.exposure import match_histograms
from skimage import color
import time
from google.colab.patches import cv2_imshow

In [ ]:
def gloablmatching(source, target,num_of_samples):
  #displaying the source and target images
  f, axArr = plt.subplots(nrows=2,ncols=3,figsize=(10,10))
  plt.sca(axArr[0,0])
  plt.imshow(source)
  plt.title('Source Image')
  
  plt.sca(axArr[0,1])
  plt.imshow(target,cmap='gray')
  plt.title('Target Image')

  #converting the RBG source image to LAB form
  LABmatrix = color.rgb2lab(source/255)

  #plotting the histogram of the l values of the source image
  plt.sca(axArr[1,0])
  plt.hist(LABmatrix[:,:,0])
  plt.title('Histogram: Source Image')

  #plotting the histogram of the intensities of the target image
  plt.sca(axArr[1,1])
  plt.hist(target[:,:])
  plt.title('Histogram: Target Image')


  #histogram matching 
  source_lvals = match_histograms(LABmatrix[:,:,0],target)
  LABmatrix[:,:,0] = source_lvals
  plt.sca(axArr[1,2])
  plt.hist(LABmatrix[:,:,0])
  plt.title('Histogram Matching: Source Image')

  #divide the image into 200 parts
  # dividing the x axis in nx=10 parts
  # and y axis in ny = 20 parts
  # take a random pixel from each part
  #The array Z stores the 200 random pixels (x,y)
  #the array Z also stores the comparison metrix: 0.5*sd + 0.5l(x,y)
  src = source
  np.random.seed(42)
  nx = 20 #number of divisions along x
  ny = np.int(num_of_samples/20) #number of divisions along y
  x = LABmatrix.shape[0]
  y = LABmatrix.shape[1]
  divx = x/nx
  divy = y/ny
  count = 0
  Z = np.zeros((num_of_samples,3))
  for i in range(nx):
    for j in range(ny):
      Z[count][0] = int(min(np.random.randint(i*divx,(i+1)*divx,1)+2,x-2))
      Z[count][1] = int(min(np.random.randint(j*divy,(j+1)*divy,1)+2,y-2))
      xcord = int(Z[count][0])
      ycord = int(Z[count][1])
      #Marking the selected random point
      src = cv2.circle(src, (ycord,xcord),radius=0,color=(255,0,0),thickness=-1)
      #the comparison metric
      Z[count][2] = 0.5*np.std(np.ravel(LABmatrix[xcord-2:xcord+2,ycord-2:ycord+2,0])) + 0.5*LABmatrix[xcord,ycord,0]
      count = count + 1

  plt.sca(axArr[0,2])
  plt.imshow(src)
  plt.title('Sampled Points')
  plt.show()

  #in the target image go through each pixel and compute the nbd stats with the available neighbors in 5x5 neighborhood
  # compare it with the 200 values of the source image and choose that value for which the mod of the difference is min
  # transfer a and b channels for the min value
  coloured_img = np.zeros((target.shape[0],target.shape[1],3))
  x = target.shape[0]
  y = target.shape[1]
  for i in range(x):
    for j in range(y):
      l = [] 
      for i1 in range(-2,3):
        for j1 in range(-2,3):
          #take the avaliable pixels for calculating the nbs stats
          if(i+i1>=0 and j+j1>=0 and i+i1<x and j+j1<y):
            l.append(target[int(i+i1),int(j+j1)])
      metric = np.std(l)*0.5 + target[i,j]*0.5
      indx = abs(Z[:,2] - metric).argmin()
      #the l value is between 0-100, hence scale it
      coloured_img[i,j,0] = target[i,j]*(100/255)
      coloured_img[i,j,1] = LABmatrix[int(Z[indx][0]),int(Z[indx][1]),1]
      coloured_img[i,j,2] = LABmatrix[int(Z[indx][0]),int(Z[indx][1]),2] 

  
  #lab image to rgb img
  result = color.lab2rgb(coloured_img)
  return result

In [ ]:
def swatchplot(src, target, swatchsrc, swatchtarget):
  i = 0
  srcimg = src.copy()
  while i <= (len(swatchsrc) - 4):
    x1 = swatchsrc[i]
    y1 = swatchsrc[i+1]
    x2 = swatchsrc[i+2]
    y2 = swatchsrc[i+3]
    srcimg = cv2.rectangle(srcimg, (x1,y1),(x2,y2), (255,0,0),1)
    i = i + 4

  targetimg = target.copy()
  i = 0
  while i <= (len(swatchtarget) - 4):
    x1 = swatchtarget[i]
    y1 = swatchtarget[i+1]
    x2 = swatchtarget[i+2]
    y2 = swatchtarget[i+3]
    targetimg = cv2.rectangle(targetimg, (x1,y1),(x2,y2), (255,0,0),1)
    i = i + 4

  print('Plotting the Swatches')
  f, axArr = plt.subplots(nrows=1,ncols=2,figsize=(7,3))
  plt.sca(axArr[0])
  plt.imshow(srcimg,aspect="auto")
  plt.title('Source Image')
  plt.sca(axArr[1])
  plt.imshow(targetimg,cmap='gray',aspect="auto")
  plt.title('Target Image')
  plt.show()
  return srcimg, targetimg

In [ ]:
def inSwatch(x,y,swatchtarget):
  indx  = 0
  while indx <= (len(swatchtarget) - 4):
    y1 = swatchtarget[indx]
    x1 = swatchtarget[indx+1]
    y2 = swatchtarget[indx+2]
    x2 = swatchtarget[indx+3]
    indx = indx + 4
    if (x >= x1 and x<=x2 and y>= y1 and y<=y2):
      return True
    
  return False;

In [ ]:
def swatchlist (swatchtarget):
  # divide the swatch into boxes of size 5 x 5
  # list l stores the centre of each such box for each swatch
  l = []
  indx = 0
  while indx <= (len(swatchtarget) - 4):
    x1 = swatchtarget[indx]
    y1 = swatchtarget[indx+1]
    x2 = swatchtarget[indx+2]
    y2 = swatchtarget[indx+3]
    indx = indx + 4
    #print(x1,y1,x2,y2)
    nx = x2 - x1
    ny = y2 - y1
    for i in range(int(nx/5)):
      for j in range(int(ny/5)):
        if (x1 + 2 + i*5 < x2 and y1 + 2 + j*5 < y2):
          l.append((x1+2+i*5,y1+2+j*5))
  
  return l

In [ ]:
def swatchtransfer(source,target,swatchsrc,swatchtarget): 
  #print('Source Image',source.shape)
  #print('Target Image',target.shape)
  #print('Source Swatches',swatchsrc)
  #print('Dest Swatches',swatchtarget)
  #print('Transferring colors between swatches')
  targetimage = np.zeros((target.shape[0],target.shape[1],3))
  targetimage[:,:,0] = np.asarray(target[:,:]*(100.0/255),dtype='int')
  #Transfer color across the swatches
  i = 0
  while i <= (len(swatchsrc) - 4):
    y1src = swatchsrc[i]
    x1src = swatchsrc[i+1]
    y2src = swatchsrc[i+2]
    x2src = swatchsrc[i+3]
    y1tar = swatchtarget[i]
    x1tar = swatchtarget[i+1]
    y2tar = swatchtarget[i+2]
    x2tar = swatchtarget[i+3]
    #print('transferring between')
    #plt.imshow(source[x1src:x2src+1,y1src:y2src+1])
    #plt.title('source swatch')
    #plt.show()
    #plt.imshow(target[x1tar:x2tar+1,y1tar:y2tar+1],cmap = 'gray')
    #plt.title('target swatch')
    #plt.show()
    result = gloablmatching(source[x1src:x2src+1,y1src:y2src+1],target[x1tar:x2tar+1,y1tar:y2tar+1],200)
    #plt.imshow(result)
    #plt.title('Swatch after matching')
    #plt.show()
    #need the result in lab format to compare intensity and luminisance
    targetimage[x1tar:x2tar+1,y1tar:y2tar+1] = color.rgb2lab(result)
    i = i + 4
  
  return  targetimage


In [ ]:
def swatches(source, target,swatchsrc, swatchtarget):
  start = time.time()
  print('Plotting Original Images')
  f, axArr = plt.subplots(nrows=1,ncols=2,figsize=(6,3))
  plt.sca(axArr[0])
  plt.imshow(source,aspect="auto")
  plt.title('Source Image')
  plt.sca(axArr[1])
  plt.imshow(target,cmap='gray',aspect="auto")
  plt.title('Target Image')
  plt.show()
  swatchplot(source, target, swatchsrc, swatchtarget)
  targetimage = swatchtransfer(source, target, swatchsrc, swatchtarget)
  print("Processing...")
  targetimage2 = np.zeros((target.shape[0],target.shape[1],3))
  targetimage2[:,:,0] = np.asarray(target[:,:]*(100.0/255) , dtype='int')
  l = swatchlist(swatchtarget)
  for i in range(targetimage.shape[0]):
    for j in range(targetimage.shape[1]):
      #if (not inSwatch(i,j,swatchtarget)):
        l1 = []
        for k in range(len(l)):
          xcord = l[k][1]
          ycord = l[k][0]
          sum = 0
          for i1 in range(-2,3):
            for j1 in range(-2,3):
              #taking a valid neighbor
              if (i+i1>= 0 and i+i1 <targetimage.shape[0] and j+j1>=0 and j+j1< targetimage.shape[1]):
                    sum = sum + (int(targetimage[xcord+i1,ycord+j1,0]) - int(targetimage2[i+i1,j+j1,0]))**2
          l1.append(sum)
      
        pos = np.argmin(l1)
        x = l[pos][1]
        y = l[pos][0]
        targetimage2[i,j,1] = targetimage[x,y,1]
        targetimage2[i,j,2] = targetimage[x,y,2]

  result1 = np.copy(targetimage)
  result2 = np.copy(targetimage2)
  #print('L channel',np.min(targetimage[:,:,0]),np.max(targetimage[:,:,0]))
  #print('a channel',np.min(targetimage[:,:,1]),np.max(targetimage[:,:,1]))
  #print('b channel',np.min(targetimage[:,:,2]),np.max(targetimage[:,:,2]))
  result_rgb1 = color.lab2rgb(result1)
  result_rgb2 = color.lab2rgb(result2)
  end = time.time()
  print('Time taken:',(end-start)/60)
  plt.imshow(result_rgb2); 
  plt.title('Final Coloured Image2')
  return result_rgb1,result_rgb2

In [ ]:
#[y1,x1,y2,x2]
source4 = plt.imread('source4.jpeg')
target4 = cv2.imread('target4.jpeg',cv2.IMREAD_GRAYSCALE)
_,result = swatches(source4, target4, [110,110,140,140,80,180,110,210],[110,110,140,140,220,145,250,175])

In [ ]:
#Only for plotting results
'''
source4 = cv2.imread('source4.jpeg')
source4 = cv2.cvtColor(source4, cv2.COLOR_BGR2RGB)
target4 = cv2.imread('target4.jpeg',cv2.IMREAD_GRAYSCALE)

srcimg, targetimg = swatchplot(source4, target4, [110,110,140,140,80,180,110,210],[110,110,140,140,220,145,250,175])
f, axArr = plt.subplots(nrows=1,ncols=3,figsize=(10,3))
plt.sca(axArr[0])
plt.imshow(srcimg,aspect='auto')
plt.title('Source Image')
plt.sca(axArr[1])

plt.imshow(targetimg,cmap='gray',aspect='auto')
plt.title('Target Image')
plt.sca(axArr[2])
plt.imshow(result,aspect='auto')
plt.title('Final coloured image')
plt.show()
'''

In [ ]:
source3 = cv2.imread('source3.png')
source3 = cv2.cvtColor(source3, cv2.COLOR_BGR2RGB)
target3 = cv2.imread('target3.png',cv2.IMREAD_GRAYSCALE)
_,result = swatches(source3, target3, [180,90,220,120],[100,40,140,70])

In [ ]:
#Only for plotting results
'''
source3 = cv2.imread('source3.png')
source3 = cv2.cvtColor(source3, cv2.COLOR_BGR2RGB)
target3 = cv2.imread('target3.png',cv2.IMREAD_GRAYSCALE)

srcimg, targetimg = swatchplot(source3, target3,[180,90,220,120],[100,40,140,70])
f, axArr = plt.subplots(nrows=1,ncols=3,figsize=(10,3))
plt.sca(axArr[0])
plt.imshow(srcimg,aspect='auto')
plt.title('Source Image')
plt.sca(axArr[1])

plt.imshow(targetimg,cmap='gray',aspect='auto')
plt.title('Target Image')
plt.sca(axArr[2])
plt.imshow(result,aspect='auto')
plt.title('Final coloured image')
plt.show()
'''

In [ ]:
source2 = cv2.imread('source2.png')
source2 = cv2.cvtColor(source2, cv2.COLOR_BGR2RGB)
target2 = cv2.imread('target2.png',cv2.IMREAD_GRAYSCALE)
_,result = swatches(source2, target2, [100,35,130,70,130,110,170,140],[10,25,40,55,130,110,170,140])

In [ ]:
#Only for plotting results
'''
source2 = cv2.imread('source2.png')
source2 = cv2.cvtColor(source2, cv2.COLOR_BGR2RGB)
target2 = cv2.imread('target2.png',cv2.IMREAD_GRAYSCALE)

srcimg, targetimg = swatchplot(source2, target2, [100,35,130,70,130,110,170,140],[10,25,40,55,130,110,170,140])
f, axArr = plt.subplots(nrows=1,ncols=3,figsize=(10,3))
plt.sca(axArr[0])
plt.imshow(srcimg,aspect='auto')
plt.title('Source Image')
plt.sca(axArr[1])

plt.imshow(targetimg,cmap='gray',aspect='auto')
plt.title('Target Image')
plt.sca(axArr[2])
plt.imshow(result,aspect='auto')
plt.title('Final coloured image')
plt.show()
'''

In [ ]:
source1 = cv2.imread('source1.png')
source1 = cv2.cvtColor(source1, cv2.COLOR_BGR2RGB)
target1 = cv2.imread('target1.png',cv2.IMREAD_GRAYSCALE)
_,result = swatches(source1, target1, [50,50,100,100,170,140,200,170], [50,50,100,100,50,140,100,170])

In [ ]:
#Only for plotting results
'''
source1 = cv2.imread('source1.png')
source1 = cv2.cvtColor(source1, cv2.COLOR_BGR2RGB)
target1 = cv2.imread('target1.png',cv2.IMREAD_GRAYSCALE)

srcimg, targetimg = swatchplot(source1, target1, [50,50,100,100,170,140,200,170], [50,50,100,100,50,140,100,170])
f, axArr = plt.subplots(nrows=1,ncols=3,figsize=(10,3))
plt.sca(axArr[0])
plt.imshow(srcimg,aspect='auto')
plt.title('Source Image')
plt.sca(axArr[1])

plt.imshow(targetimg,cmap='gray',aspect='auto')
plt.title('Target Image')
plt.sca(axArr[2])
plt.imshow(result,aspect='auto')
plt.title('Final coloured image')
plt.show()'''

In [ ]:
#codes to get swatches interactively from the user
def onclick(event): 
    print('clicked')
    global coordinates
    if(len(coordinates)>=2):
        coordinates = []
    coordinates.append((event.xdata, event.ydata))
    global ax
    plt.plot(event.xdata, event.ydata, marker='v', color="red")
    #print("button=%f, x=%f, y=%f, xdata=%f, ydata=%f" % ( 
    #     event.button, event.x, event.y, event.xdata, event.ydata)) 

In [ ]:
#select swatch function
def selectSourceSwatch(source):
    %matplotlib notebook
    print('Select Source Swatch')
    coordinates = []
    fig, ax = plt.subplots(figsize=(10, 6))

    plt.imshow(source)
    cid = fig.canvas.mpl_connect('button_press_event', onclick) 
    plt.show() 
    swatchsrc = coordinates
    return swatchsrc
    
def selectTargetSwatch(target):
    %matplotlib notebook
    print('Select Target Swatch')
    coordinates = []
    fig, ax = plt.subplots(figsize=(10, 6))

    plt.imshow(target)
    cid = fig.canvas.mpl_connect('button_press_event', onclick) 
    plt.show() 
    
    swatchtarget = coordinates
    return swatchtarget  

In [ ]:
#img = plt.imread("Documents\A1images1\A1images\source4.jpeg")
#selectSourceSwatch(img)

In [ ]:
#swatchsrc = coordinates
#print(swatchsrc)

In [ ]:
#selectTargetSwatch(img)

In [ ]:
#swatchTarget = coordinates
#print(swatchTarget)